# Lab 1: Local feature detection
This is a notebook, which could help you with testing second lab assignment.
It contains utility functions for visualization, some test input for the functions you needs to implement,
and the output of the reference solution for the same test input.

template functions for the assignment contain a short description of what the function is supposed to do,
and produce an incorrect output, which is nevertheless in proper format: type and shape.

You are not allowed to use kornia or opencv or any other library functions, which are specifically designed
to perform the operations requested in assignment

In [ ]:
# % load_ext autoreload
# % autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import torch
import kornia
import cv2


# Visualization functions
def plot_torch(x, y, *kwargs):
    plt.plot(x.detach().cpu().numpy(), y.detach().cpu().numpy(), *kwargs)
    return


def imshow_torch(tensor, figsize=(8, 6), *kwargs):
    plt.figure(figsize=figsize)
    plt.imshow(kornia.tensor_to_image(tensor), *kwargs)
    return


def imshow_torch_channels(tensor, dim=1, *kwargs):
    num_ch = tensor.size(dim)
    fig = plt.figure(figsize=(num_ch * 5, 5))
    tensor_splitted = torch.split(tensor, 1, dim=dim)
    for i in range(num_ch):
        fig.add_subplot(1, num_ch, i + 1)
        plt.imshow(kornia.tensor_to_image(tensor_splitted[i].squeeze(dim)),
                   *kwargs)
    return


def timg_load(fname, to_gray=True):
    img = cv2.imread(fname)
    with torch.no_grad():
        timg = kornia.image_to_tensor(img, False).float()
        if to_gray:
            timg = kornia.color.bgr_to_grayscale(timg)
        else:
            timg = kornia.color.bgr_to_rgb(timg)
    return timg


img_blobs = timg_load('sin.png')

# Source https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_corner.html
img_corners = timg_load('corners.png')

imshow_torch(img_blobs)
imshow_torch(img_corners)

In [ ]:
from local_detector import hessian_response

resp_small = hessian_response(img_blobs, 1.6)
resp_big = hessian_response(img_blobs, 25.)

imshow_torch_channels(torch.cat([resp_small, resp_big], dim=0), 0)

## Reference example

```python
from local_detector import hessian_response

resp_small = hessian_response(img_blobs, 1.6)
resp_big = hessian_response(img_blobs, 25.)

imshow_torch_channels(torch.cat([resp_small,
                                 resp_big], dim=0), 0)
```
![image.png](local_detector_files/att_00000.png)

In [ ]:
from local_detector import harris_response

resp_small = harris_response(img_corners, 1.6, 2.0, 0.04)
resp_big = harris_response(img_corners, 7., 9., 0.04)

imshow_torch_channels(torch.cat([resp_small,
                                 resp_big], dim=0), 0)


## Reference example

```python
from local_detector import harris_response

resp_small = harris_response(img_corners, 1.6, 2.0, 0.04)
resp_big = harris_response(img_corners, 7., 9., 0.04)

imshow_torch_channels(torch.cat([resp_small, 
                                 resp_big], dim=0), 0)
```
![image.png](local_detector_files/att_00001.png)

In [ ]:
from local_detector import nms2d
    
nmsed_harris = nms2d(resp_small, 0.00004)
imshow_torch(nmsed_harris)


## Reference example

```python
from local_detector import nms2d

nmsed_harris = nms2d(resp_small, 0.00004)

imshow_torch(nmsed_harris)
```
![image.png](local_detector_files/att_00002.png)

In [ ]:
from local_detector import harris

keypoint_locations = harris(img_corners, 1.6, 2.0, 0.00001)
print(keypoint_locations)


## Reference example

```python
from local_detector import harris

keypoint_locations = harris(img_corners, 1.6, 2.0, 0.00001)
print (keypoint_locations)
```

    tensor([[  0,   0,  21,  32],
            [  0,   0,  24,  55],
            [  0,   0,  28,  86],
            [  0,   0,  31, 110],
            [  0,   0,  31, 201],
            [  0,   0,  31, 248],
            [  0,   0,  44,  42],
            [  0,   0,  46,  33],
            [  0,   0,  48,  49],
            [  0,   0,  48,  70],
            [  0,   0,  51,  97],
            [  0,   0,  52,  77],
            [  0,   0,  55, 124],
            [  0,   0,  56, 104],
            [  0,   0,  70,  48],
            [  0,   0,  71,  60],
            [  0,   0,  75,  87],
            [  0,   0,  76,  67],
            [  0,   0,  78, 201],
            [  0,   0,  79,  94],
            [  0,   0,  79, 115],
            [  0,   0,  79, 249],
            [  0,   0,  81, 298],
            [  0,   0,  83, 122],
            [  0,   0,  86, 144],
            [  0,   0,  99,  78],
            [  0,   0, 101,  68],
            [  0,   0, 102, 105],
            [  0,   0, 103,  85],
            [  0,   0, 106, 132],
            [  0,   0, 107, 112],
            [  0,   0, 109, 159],
            [  0,   0, 110, 139],
            [  0,   0, 114,  76],
            [  0,   0, 115,  88],
            [  0,   0, 120, 120],
            [  0,   0, 122, 143],
            [  0,   0, 128, 251],
            [  0,   0, 128, 298],
            [  0,   0, 160,  78],
            [  0,   0, 160, 272]])

In [ ]:
from local_detector import create_scalespace
# from imagefiltering import gaussian_filter2d

pyr, sigmas = create_scalespace(img_blobs, 10, 1.6)

resps = torch.stack([hessian_response(x.squeeze(2), 3.6) for x in torch.split(pyr, 1, 2)], dim=2)

imshow_torch_channels(pyr, 2)
imshow_torch_channels(resps, 2)


## Reference example

```python
from local_detector import create_scalespace
pyr, sigmas = create_scalespace(img_blobs, 10, 1.6)
print (pyr.shape)
resps = torch.stack([hessian_response(x.squeeze(2), 3.6) for x in torch.split(pyr,1,2)], dim=2)
imshow_torch_channels(pyr, 2)
imshow_torch_channels(resps, 2)
```
    
    torch.Size([1, 1, 10, 256, 256])

![image.png](local_detector_files/att_00003.png)

In [ ]:
from local_detector import create_scalespace
pyr, sigmas = create_scalespace(img_corners, 10, 1.3)
print (pyr.shape)
resps = torch.stack([harris_response(x.squeeze(2), 3.6, 4.0, 0.04) for x in torch.split(pyr,1,2)], dim=2)
imshow_torch_channels(pyr, 2)
imshow_torch_channels(resps, 2)

## Reference solution

```python
from local_detector import create_scalespace
pyr, sigmas = create_scalespace(img_corners, 10, 1.3)
print (pyr.shape) 
resps = torch.stack([harris_response(x.squeeze(2), 3.6, 4.0, 0.04) for x in torch.split(pyr,1,2)], dim=2)
imshow_torch_channels(pyr, 2)
imshow_torch_channels(resps, 2)
```

![image.png](local_detector_files/att_00004.png)

In [ ]:
# from local_detector import scalespace_hessian
from local_detector import *

def visualize_detections(img, keypoint_locations, img_idx=0,
                         increase_scale=1.):
    # Select keypoints relevant to image   
    kpts = [cv2.KeyPoint(b_ch_sc_y_x[4].item(),
                         b_ch_sc_y_x[3].item(),
                         b_ch_sc_y_x[2].item())
            for b_ch_sc_y_x in keypoint_locations if
            b_ch_sc_y_x[0].item() == img_idx]
    vis_img = None
    vis_img = cv2.drawKeypoints(
            kornia.tensor_to_image(img).astype(np.uint8),
            kpts,
            vis_img,
            flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.figure(figsize=(12, 10))
    plt.imshow(vis_img)
    return


with torch.no_grad():
    keypoint_locations = scalespace_hessian(img_blobs / img_blobs.max(), 0.003)
    
visualize_detections(img_blobs, keypoint_locations, increase_scale=1.0)

## Reference example

```python
from local_detector import scalespace_hessian
def visualize_detections(img, keypoint_locations, img_idx = 0, increase_scale = 1.):
    # Select keypoints relevant to image   
    kpts = [cv2.KeyPoint(b_ch_sc_y_x[4],b_ch_sc_y_x[3],b_ch_sc_y_x[2])
            for b_ch_sc_y_x in keypoint_locations if b_ch_sc_y_x[0] == img_idx]
    vis_img = None
    vis_img = cv2.drawKeypoints(kornia.tensor_to_image(img).astype(np.uint8),
                                kpts,
                                vis_img, 
                                flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.figure(figsize=(12,10))
    plt.imshow(vis_img)
    return


with torch.no_grad():
    keypoint_locations = scalespace_hessian(img_blobs/img_blobs.max(), 0.003)

visualize_detections(img_blobs, keypoint_locations, increase_scale=1.0)
```
![image.png](local_detector_files/att_00005.png)

In [ ]:
from local_detector import scalespace_hessian


def visualize_detections(img, keypoint_locations, img_idx=0,
                         increase_scale=1.):
    # Select keypoints relevant to image   
    kpts = [cv2.KeyPoint(b_ch_sc_y_x[4].item(),
                         b_ch_sc_y_x[3].item(),
                         increase_scale * b_ch_sc_y_x[2].item())
            for b_ch_sc_y_x in keypoint_locations if
            b_ch_sc_y_x[0].item() == img_idx]

    vis_img = None
    vis_img = cv2.drawKeypoints(
            kornia.tensor_to_image(img).astype(np.uint8),
            kpts,
            vis_img,
            flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.figure(figsize=(12, 10))
    plt.imshow(vis_img)
    return


img_sun = timg_load('sunflowers.png', True)
with torch.no_grad():
    keypoint_locations = scalespace_hessian(img_sun / img_sun.max(), 0.02)

visualize_detections(img_sun, keypoint_locations, increase_scale=3.0)

```python
from local_detector import scalespace_hessian
def visualize_detections(img, keypoint_locations, img_idx = 0, increase_scale = 1.):
    # Select keypoints relevant to image   
    kpts = [cv2.KeyPoint(b_ch_sc_y_x[4],b_ch_sc_y_x[3],increase_scale*b_ch_sc_y_x[2])
            for b_ch_sc_y_x in keypoint_locations if b_ch_sc_y_x[0] == img_idx]
    
    vis_img = None
    vis_img = cv2.drawKeypoints(kornia.tensor_to_image(img).astype(np.uint8),
                                kpts,
                                vis_img, 
                                flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.figure(figsize=(12,10))
    plt.imshow(vis_img)
    return

img_sun = timg_load('sunflowers.png', True)
with torch.no_grad():
    keypoint_locations = scalespace_hessian(img_sun/img_sun.max(), 0.02)

visualize_detections(img_sun, keypoint_locations, increase_scale=3.0)
```
![image.png](local_detector_files/att_00006.png)

In [ ]:
# from local_detector import scalespace_harris
from local_detector import harris_response
from imagefiltering import gaussian_filter2d

with torch.no_grad():
    keypoint_locations = scalespace_harris(img_corners, 0.00001)

visualize_detections(img_corners * 255., keypoint_locations,
                     increase_scale=1.0)

## Reference example

```python
from local_detector import scalespace_harris
from imagefiltering import gaussian_filter2d

with torch.no_grad():
    keypoint_locations = scalespace_harris(img_corners, 0.00001)

visualize_detections(img_corners*255., keypoint_locations, increase_scale=1.0)
```
![image.png](local_detector_files/att_00007.png)